In [71]:
pip install haystack-ai

In [72]:
!pip install "sentence-transformers>=3.0.0" "huggingface_hub>=0.23.0"

In [73]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret
from haystack.components.generators import HuggingFaceAPIGenerator

In [74]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path

In [75]:
#Preprocessing-Pipeline bauen
document_store = InMemoryDocumentStore()
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
document_joiner = DocumentJoiner()

In [76]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)

In [77]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

In [78]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

In [79]:
preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (List[Document])
  - document_cleaner.documents -> document_splitter.documents (List[Document])
  - document_splitter.documents -> document_embedder.documents (List[Document])
  - document_embedder.documents -> document_writer.documents (List[Document])

In [80]:
#from sentence_transformers import SentenceTransformer

In [81]:
#pip uninstall sentence-transformers

In [82]:
#pip install "sentence-transformers>=3.0.0"

In [83]:
output_dir = "/content/"  # Pfad zum Verzeichnis, wo sich die Datei befindet
data_file = Path(output_dir) / "dump.txt"  # der Dateiname
preprocessing_pipeline.run({"file_type_router": {"sources": [data_file]}})

Batches:   0%|          | 0/6093 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 194946}}

In [84]:
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

In [85]:
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)

In [86]:
#token bei Hugging-Face unter Access-Token erstellen, wichtig: write!
generator = HuggingFaceAPIGenerator(api_type="serverless_inference_api",
                                    api_params={"model": "mistralai/Mixtral-8x7B-Instruct-v0.1"},
                                    token=Secret.from_token("<add_your_hf_api_token_here>"))

In [87]:
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [88]:
question = '''In One Piece, does a Devil Fruit reappear after the user dies?

In chapter 702, we see Ace's Devil Fruit 'Mera Mera no Mi' being showcased as the prize of the event held in the Colosseum. The announcer of the event mentions that the "fruit is reborn."

Does it mean every Devil Fruit has that ability to be reborn once the user dies, or is it an artificial Devil Fruit made by Doflamingo's factory?'''

In [89]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [90]:
print(results["llm"]["replies"])

[' According to the One Piece wiki, Devil Fruits do exist in a cycle of reincarnation. When a Devil Fruit user dies, its ability is reborn into a new Devil Fruit in a random part of the world, and the one who eats it will become the new user. This is true for both natural and artificial Devil Fruits. The Mera Mera no Mi is a natural Devil Fruit, so it has the ability to be reborn after the user dies.']


In [91]:
question = '''Does Devil's Fruit power gained depend on how much of it is eaten?

Luffy accidentally ate the Gomu-Gomu (Rubber-Rubber) fruit, but was interrupted by Shanks after a few bites. It was too late and Luffy got the Devil's Fruit powers anyway. If he had instead eaten the whole of it, would he have got even more power from the fruit?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" No, the amount of Devil's Fruit eaten does not affect the power gained. Once a person consumes a Devil's Fruit, they gain its powers for the rest of their life, regardless of how much of the fruit they ate. This is stated in the documents provided."]


In [92]:
question = '''Can multiple people/animals eat a single Devil Fruit? Provided, Chopper eats the Devil Fruit along with the other deers, will they all have the Devil Fruit ability, given that acquiring the ability only requires a single bite?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" No, multiple people/animals cannot eat a single Devil Fruit. Once someone takes a single bite out of the fruit, the ability becomes solely theirs in that moment, and from then on, the fruit simply becomes a insanely gross-tasting fruit. Still, everyone takes care to eat the whole thing without knowing this. This is because the Devil Fruit Abilities are transferred after one bite. After that, it's just a nasty old fruit. In the instance that they take a bite at the same time, the power will belong to the one who swallows first. Nothing will happen if anyone else takes a bite. So in summation, a single fruit cannot give its powers to multiple people."]


In [93]:
question = '''What will happen if one person eats two Devil Fruits? If a person has already eaten one Devil Fruit, what will happen to him if he eats another Devil Fruit?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" According to Blueno, a person will explode if he eats two Devil Fruits. This has been proven to be true based on the reactions of pirates and Marines to Blackbeard's power usage. However, Blackbeard didn't actually eat two Devil Fruits, but acquired the power from another person."]


In [94]:
question = '''How is the ability of a Devil Fruit disovered? Since there is just one Devil Fruit for a specific ability, how does one know about what ability it holds?
For example, Buggy and Teach knew about the powers their Devil Fruits would have provided them.
It is mentioned that they had read about it.
But if the fruit occurs only once till the user dies, how did they find out about the fruits' powers?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[' The ability of a Devil Fruit is discovered when it is eaten and the power is manifested. However, there are also resources such as the Devil Fruit Encyclopedia that provide information about the names and powers of Devil Fruits. It is mentioned that Buggy and Teach had read about the powers their Devil Fruits would provide them, implying that they had access to such resources. Additionally, some Devil Fruits have a distinct shape that can give a clue about their abilities, but this is not always the case. It is also possible for a sufficiently skilled scientist to create an extract of a Devil Fruit and administer it to an infant who cannot yet eat solid food, resulting in the recipient gaining the ability as normal while the fruit becomes worthless.']


In [95]:
question = '''What did Enel do after being defeated? As shown in the Manga and the Anime, Enel survived along with the Ark. He then Embarks to Fairy Verth

In the Manga it shows his adventures on the Moon. But what happened? I didn't understand the situation that happened. What exactly happened on the Moon?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[' After being defeated by Luffy, Enel headed off to Fairy Vearth, which turns out to be the Moon. There he found a group of alien-like creatures—who have wings similar to the Skypieans—which Enel befriended. He also discovered the history of the Skypieans, Shandians, and Birkans, who all originated from the moon city and invented the automata. After using up their resources, the inhabitants decided to travel to Earth while giving a tearful goodbye to their creations. The automata then pledged their loyalty to Enel for reawakening them. Enel accepted their loyalty and stood tall as the leader of his new army.']


In [96]:
question = '''How do Luffy's Gear attacks work? Might just be me missing an explanation part in the manga, or just not properly understanding at the time. But how does Luffy's Gear 2nd and 3rd attacks work? '''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" Luffy's Gear 2nd and 3rd attacks work by him increasing the size and power of his body. Gear 2nd is achieved by Luffy increasing the blood flow to his muscles, making him faster and stronger but also causing him to use up his stamina quickly. Gear 3rd is achieved by Luffy blowing air into his bones, making them expand and increasing his physical size and strength. However, this also makes him slower and less maneuverable. Both of these forms are activated by Luffy biting down on his thumb and blowing air into it, creating a distinctive sound."]


In [97]:
question = '''How does the One piece Bounty system work?

Right from the beginning of One piece, we see luffy and his crew keep getting a bounty increase everytime something big happens and this leaves a lot of room for pondering; especially on how these bounties are thought up and agreed on. I understand that Pirates who are deemed to be dangerous by the Marines are awarded bounties but my question is how the bounty system actually works in relation to the media. Do the Marines kinda depend on information from newspaper media outlets to increase or award a bounty? (or is the other way round; that they only use newspaper and the media for getting the bounties out). Is the Bounty based more on popularity within the media?

Also, what warrants a decrease/withdrawal in Bounty? I know it has never technically happened but is it ideally possble?

My last question is how a pirate with a bounty can actually turn in another pirate for their bounty, are there any set restrictions on awarding Bounties?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" The One Piece bounty system is primarily based on the perceived threat level of a pirate to the World Government and the Marines. The bounty is determined by a committee within the Marines, and is influenced by various factors such as the pirate's strength, the number of people they have harmed, and the amount of damage they have caused. The media, including newspapers, do play a role in the bounty system, as they help to spread information about pirates and their crimes, which can influence the Marines' decision on the bounty amount. However, the bounty is not based solely on popularity within the media.\n\nA decrease or withdrawal in bounty is ideally possible, but it is rare and usually only happens in certain circumstances. For example, if a pirate turns themselves in and agrees to work for the World Government, their bounty may be withdrawn or reduced. Additionally, if a pirate is believed to be dead, their bounty may be withdrawn. However, it is important to note that the Mari

In [98]:
question = '''Why does a Devil Fruit's power nullify only if the user is knocked unconscious and not when he sleeps?

I have noticed something rather strange: Why do devil fruit users abilities nullify when they are knocked unconscious but not when they fall sleep?

Take Moria for example. Moria had control of all the shadows when Luffy knocked him out, allowing the shadows to escape, so why does sleeping not do the same? Certainly he had to have fallen asleep at some point in time before Luffy had knocked him out.

Same is the case with Sugar. She was scared unconscious by Usopp and all the toys turned back to their human forms. Is there something about being knocked unconscious and falling asleep that is different in the One Piece Universe?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" In the One Piece universe, being knocked unconscious and falling asleep have different effects on Devil Fruit users. When a Devil Fruit user is knocked unconscious, their powers are nullified, meaning they lose control over their abilities. This is because being knocked unconscious is often the result of a physical trauma, which can disrupt the user's connection to their Devil Fruit power.\n\nOn the other hand, falling asleep does not have the same effect on Devil Fruit users. This is because sleep is a natural process that does not involve any physical trauma, so the user's connection to their Devil Fruit power remains intact.\n\nIn the case of Moria, he was knocked unconscious by Luffy, which caused him to lose control over the shadows he had absorbed. If Moria had simply fallen asleep, his powers would not have been affected.\n\nSimilarly, when Sugar was scared unconscious by Usopp, she lost control over her powers, causing all the toys to turn back to their human forms. If Sugar

In [99]:
question = '''Has Luffy ever killed anyone?  In One Piece, there is a lot of blood and fights as we saw on the death of Ace.

However, did Luffy ever actually kill any person?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[' No, Luffy has not killed anyone in One Piece. He has been involved in many fights and battles, but he has never taken a life. This is consistent with his character as a pirate who values freedom and the lives of others.']


In [100]:
question = '''Are the clothes of Devil Fruit users made of a special material? We have Akainu, who is a Logia user with magma capabilities, and Luffy, who can stretch with the Gomu Gomu no Mi. Are their clothes made up of some special material that doesn't burn or tear apart? '''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" Yes, the clothes of Devil Fruit users are made of a special material that doesn't burn or tear apart. This is because the clothes of Devil Fruit users are not made of regular materials, but rather, they are made of the same substance as the user's Devil Fruit power. For example, Akainu's clothes are made of magma, and Luffy's clothes are made of rubber. This is why their clothes do not burn or tear apart when they use their Devil Fruit powers."]


In [ ]:
question = '''What are the names and grades of Zoro's swords? Zoro always (almost always) carries 3 swords with him. So the question is, what are the names and the grades of those 3 swords he is currently carrying?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

In [101]:
question = '''Is there a re-encounter with the sea monster that ate Shanks's arm? Just of out curiosity, do we ever get to see the same sea monster that ate Shanks's arm way back in the beginning of One Piece?

I have only kept up with the series until Impel Down, so please no spoilers beyond a "yes/no they did/did not re-encounter the sea monster"'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" Yes, the same sea monster that ate Shanks's arm re-encounters the Straw Hat Pirates in the Fish-Man Island arc. However, it is not explicitly stated that it is the same sea monster, but it is heavily implied due to its unique appearance and abilities."]


In [102]:
question = '''How can CP9 members and Sanji run in the air? How can CP9 members and Sanji(post-timeskip) run and stand in the air? Do they use something like Haki? If not, what kind of techniques do they use? If it is Haki, can Luffy and Zoro run and stand like they do?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[' CP9 members and Sanji can run and stand in the air using their Rokushiki techniques, specifically Soru and Geppo. Soru allows them to move at high speeds by kicking off the air, while Geppo allows them to jump and stand in the air by kicking off the air. These techniques do not involve Haki, but rather involve the precise control and manipulation of their own body movements and the surrounding air. Luffy and Zoro do not possess these techniques, but they have their own unique abilities and techniques that allow them to fight effectively in various situations.']


In [ ]:
question = '''How did Luffy get the scar on his chest? There's a debate going on how Luffy got the scar on his chest but I recently seen episode 223 where Zoro and Luffy we're fighting and Zoro gave him that scar. Then for what reason is there a debate on how he got his scar?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

In [103]:
question = '''What is Fairy Vearth? The 'Fairy Vearth' that God Enel keeps talking about, the continuous land. Is he talking about the moon?'''
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)
print(results["llm"]["replies"])

[" Yes, Fairy Vearth is the Moon. This is confirmed in the Skypiea Arc when Enel reaches the Moon and finds a group of alien-like creatures there. The name 'Fairy Vearth' is a translation issue, as both 'Endless' and 'Fairy' are written in the manga, with the Kanji listing it as 'Endless Earth' and the Furigana reading as 'Fairy Vearth'."]
